# **ГЛАВА 1 – ИНИЦИАЛИЗАЦИЯ ПРОЕКТА**

## SAE for CLIP – Полный пайплайн проекта

Ноутбук выполняет полный цикл работы:

1. Подключение дисков  
2. Установка зависимостей  
3. Проверка данных  
4. Загрузка активаций и моделей  
5. Оценка качества  
6. Интерпретация SAE-фич  
7. Формирование отчётов

## 1_Импорты

In [ ]:
# Базовые библиотеки, которые нужны для оркестрации.
import os
import shutil
import subprocess
from pathlib import Path

## 2_Подключение дисков

*   Подключение Google Drive
*   Подключение Яндекс.Диска



In [ ]:
# ====== ПОДКЛЮЧЕНИЕ GOOGLE DRIVE ======
from google.colab import drive
drive.mount('/content/drive')

# ====== ПОДКЛЮЧЕНИЕ YANDEX DISK (READ-ONLY) ======
import os
import getpass

YANDEX_MOUNT = "/content/yadisk"

if not os.path.exists(YANDEX_MOUNT):
    os.makedirs(YANDEX_MOUNT)

print("Подключение Яндекс.Диска через WebDAV...")

!apt-get -y install davfs2

webdav_url = "https://webdav.yandex.ru"

!mount -t davfs https://webdav.yandex.ru /content/yadisk -o ro

print("\nДиски успешно подключены:")
print("Google Drive: /content/drive/MyDrive")
print("Yandex Disk:  /content/yadisk")


## 3_Инициализация структуры каталогов

### Глобальные пути проекта

In [ ]:
from pathlib import Path

GOOGLE_ROOT = Path("/content/drive/MyDrive/SAE_PROJECT")
YANDEX_ROOT = Path("/content/yadisk/SAE_PROJECT")

(GOOGLE_ROOT / "datasets").mkdir(parents=True, exist_ok=True)
(GOOGLE_ROOT / "activations").mkdir(parents=True, exist_ok=True)
(GOOGLE_ROOT / "models").mkdir(parents=True, exist_ok=True)
(GOOGLE_ROOT / "results").mkdir(parents=True, exist_ok=True)
(GOOGLE_ROOT / "logs").mkdir(parents=True, exist_ok=True)

print("Структура каталогов создана")

### Подключение кода проекта

In [ ]:
import sys

PROJECT_CODE = "/content/drive/MyDrive/clip-sae-interpret/clip-sae-interpret_clean"
if PROJECT_CODE not in sys.path:
    sys.path.append(PROJECT_CODE)

print("Код проекта подключен")

# ГЛАВА 2 – УСТАНОВКА И ПРОВЕРКА СРЕДЫ

## 1_Установка зависимостей

In [ ]:
print("Установка зависимостей проекта...")

!pip install -r {PROJECT_CODE}/requirements.txt

import torch
print("\nПроверка CUDA:")
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))

# Базовые библиотеки
import os
import shutil
import subprocess
from pathlib import Path

print("\nЗависимости успешно установлены")

## 2_Установка пакета проекта

In [ ]:
print("Установка пакета проекта в режиме разработки...")

%cd {PROJECT_CODE}
!pip install -e .

print("Пакет проекта установлен")

## 3_Загрузка конфигурации

In [ ]:
import yaml

with open(f"{PROJECT_CODE}/configs/train_config.yaml") as f:
    cfg = yaml.safe_load(f)

print("Конфигурация загружена")


## 4_Проверка импортов проекта


In [ ]:
from sae_clip.models.clip_wrapper import CLIPWrapper
from sae_clip.data.datasets import CIFAR10ZeroShotDataset, Food101ZeroShotDataset

print("Импорты OK")

# ГЛАВА 3 – РАБОТЫ С ДАННЫМИ

## 1_Вспомогательные функции работы с файлам

In [ ]:
import os, shutil, subprocess

def exists_local(path):
    return os.path.exists(path)


def activations_filename(name):
    if name == "cifar10":
        return "activations_cifar10_vit_b32.pt"
    if name == "food101":
        return "activations_food101_vit_b32.pt"
    return f"activations_{name}.pt"


def dataset_dirname(name):
    if name == "cifar10":
        return "cifar-10-batches-py"
    if name == "food101":
        return "food-101"
    return name


def model_dirname(name):
    if name == "cifar10":
        return "sae_cifar_512d"
    if name == "food101":
        return "sae_food101_512d"
    return f"sae_{name}_512d"


def ensure_dataset(name):
    dname = dataset_dirname(name)

    local = GOOGLE_ROOT / "datasets" / dname
    yandex = YANDEX_ROOT / "datasets" / dname

    print(f"\nПроверка датасета: {name}")

    if exists_local(local):
        print(f"Датасет найден в Google Drive: {local}")
        return local

    if exists_local(yandex):
        print(f"Датасет найден на Яндекс.Диске: {yandex}")
        print("Использование датасета напрямую с Яндекс.Диска")
        return yandex

    print("Датасет не обнаружен")
    print("Начинается скачивание датасета...")

    subprocess.run([
        "python",
        f"{PROJECT_CODE}/scripts/download_dataset.py",
        name,
        str(local)
    ])

    print(f"Датасет загружен: {local}")
    return local


def ensure_activations(name):
    fname = activations_filename(name)

    local = GOOGLE_ROOT / "activations" / fname
    yandex = YANDEX_ROOT / "activations" / fname

    print(f"\nПроверка CLIP-активаций: {name}")

    if exists_local(local):
        print(f"Активации найдены в Google Drive: {local}")
        return local

    print("Активации не найдены в Google Drive")

    if exists_local(yandex):
        print(f"Активации найдены на Яндекс.Диске: {yandex}")
        print("Копирование активаций в Google Drive...")

        local.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(yandex, local)

        print(f"Активации скопированы: {local}")
        return local

    print("Активации не обнаружены")
    print("Начинается извлечение активаций...")

    if name == "food101":
        subprocess.run(["python", f"{PROJECT_CODE}/scripts/extract_clip_activations_food101.py"])
    else:
        subprocess.run(["python", f"{PROJECT_CODE}/scripts/extract_clip_activations_cifar10.py"])

    print(f"Активации созданы: {local}")
    return local


def ensure_model(name):
    dname = model_dirname(name)

    local = GOOGLE_ROOT / "models" / dname
    yandex = YANDEX_ROOT / "models" / dname

    print(f"\nПроверка модели SAE: {name}")

    if exists_local(local):
        print(f"Модель найдена в Google Drive: {local}")
        return local

    print("Модель не найдена в Google Drive")

    if exists_local(yandex):
        print(f"Модель найдена на Яндекс.Диске: {yandex}")
        print("Копирование модели в Google Drive...")

        shutil.copytree(yandex, local, dirs_exist_ok=True)

        print(f"Модель скопирована: {local}")
        return local

    print("Модель не обнаружена")
    print("Начинается обучение модели SAE...")

    subprocess.run(["python", f"{PROJECT_CODE}/scripts/train_sae.py", name])

    print(f"Модель обучена: {local}")
    return local


# ГЛАВА 4 – ЗАГРУЗКА ДАТАСЕТОВ И АКТИВАЦИЙ

## 1_Загрузка датасетов

In [ ]:
from pathlib import Path

GOOGLE_ROOT = Path("/content/drive/MyDrive/SAE_PROJECT")
YANDEX_ROOT = Path("/content/yadisk/SAE_PROJECT")
PROJECT_CODE = Path("/content/drive/MyDrive/clip-sae-interpret/clip-sae-interpret_clean")

DATASETS = ["cifar10", "food101"]

In [ ]:
from sae_clip.data.datasets import CIFAR10ZeroShotDataset, Food101ZeroShotDataset

print("Загрузка датасетов...")

datasets = {}

for name in DATASETS:
    path = ensure_dataset(name)

    print(f"\nИнициализация датасета {name} из: {path}")

    if name == "cifar10":
        datasets[name] = CIFAR10ZeroShotDataset(root=str(path))

    elif name == "food101":
        # для torchvision Food101 root должен быть каталог, содержащий папку food-101
        parent_root = str(path.parent)

        print("Используем root для Food101:", parent_root)

        datasets[name] = Food101ZeroShotDataset(root=parent_root)

    print(f"Датасет {name} успешно загружен")

print("\nВсе датасеты готовы")


## 2_Загрузка CLIP-активаций

### Cоздание активаций для CIFAR-10
- делал его на Kaggle и загрузи его на яндекс диск

In [ ]:
# Извлечь активации 512D
'''
!python scripts/extract_clip_activations_cifar10.py \
  --data_root /content/yadisk/SAE_PROJECT/datasets \
  --output_path /content/drive/MyDrive/SAE_PROJECT/activations/activations_cifar10_vit_b32.pt \
  --train_split
'''

### Cоздание активаций для food-101
- делал его на Kaggle и загрузи его на яндекс диск

In [ ]:
# Извлечь активации 512D
'''
!python scripts/extract_clip_activations_food101_fixed.py \
  --images_dir datasets/food-101/images \
  --output_path /content/drive/MyDrive/SAE_PROJECT/activations/activations_food101_vit_b32.pt \
  --max_images 101000
'''

In [ ]:
import torch

print("Загрузка CLIP-активаций...")

activations = {}

for name in DATASETS:
    path = ensure_activations(name)

    print(f"Загрузка активаций из файла: {path}")
    data = torch.load(path)

    activations[name] = data
    print(f"Активации {name} успешно загружены, shape: {data.shape}")

print("Все активации готовы")


## 3_Загрузка моделей SAE

In [ ]:
# Обучение cifar-10 (можно сделать на kaggle) - запускается если нет на дисках
!python scripts/train_sae_google_only.py \
  --dataset cifar10 \
  --epochs 50 \
  --batch_size 256 \
  --lr 0.0005 \
  --l1_coef 0.001 \
  --latent_dim 4096

In [ ]:
# Обучение food-101 (можно сделать на kaggle) - запускается если нет на дисках
!python scripts/train_sae_google_only.py \
  --dataset food101 \
  --epochs 50 \
  --batch_size 256 \
  --lr 0.0005 \
  --l1_coef 0.001 \
  --latent_dim 4096

In [ ]:
import torch
from sae_clip.models.sae import SparseAutoencoder
import re

print("Загрузка моделей SAE...")

models = {}

for name in DATASETS:
    path = ensure_model(name)

    print(f"\nЗагрузка модели из каталога: {path}")

    # поиск всех файлов вида sae_epoch_XX.pt
    pt_files = [f for f in os.listdir(path) if f.endswith(".pt")]

    if not pt_files:
        raise FileNotFoundError(f"Не найден ни один файл .pt в каталоге: {path}")

    # извлечение номера эпохи и сортировка
    def get_epoch(fname):
        m = re.search(r"epoch_(\d+)", fname)
        return int(m.group(1)) if m else -1

    pt_files = sorted(pt_files, key=get_epoch)

    # выбор последнего (максимальная эпоха)
    weight_file = os.path.join(path, pt_files[-1])

    print(f"Найдено файлов: {pt_files}")
    print(f"Выбран последний чекпоинт: {weight_file}")

    model = SparseAutoencoder(input_dim=512, latent_dim=4096)

    state = torch.load(weight_file, map_location="cpu")
    model.load_state_dict(state)

    models[name] = model

    print(f"Модель {name} успешно загружена из {pt_files[-1]}")

print("\nВсе модели готовы")


## 4_Проверка работы моделей

In [ ]:
print("Проверка работы моделей...")

for name in DATASETS:
    x = activations[name][:10]

    # корректное приведение типов
    x = x.detach().clone().float()

    x_hat, z = models[name](x)

    print(f"{name}: вход {x.shape} → выход {x_hat.shape}")

print("Тестовый инференс выполнен успешно")


## 5_Тестовый инференс моделей

In [ ]:
print("Тестовый инференс моделей...")

for name in DATASETS:
    x = activations[name][:10]

    # корректное приведение типов
    x = x.detach().clone().float()

    x_hat, z = models[name](x)

    print(f"{name}: вход {x.shape} → выход {x_hat.shape}")

print("Тестовый инференс выполнен успешно")


## 6_Расчёт метрик SAE

In [ ]:
import torch
from sae_clip.models.sae import SparseAutoencoder

print("Расчёт метрик моделей...\n")

metrics = {}

for name in DATASETS:
    print(f"Оценка для датасета: {name}")

    x = activations[name].detach().clone().float()

    model = models[name]
    x_hat, z = model(x)

    mse = SparseAutoencoder.reconstruction_loss(x, x_hat).item()
    l1 = SparseAutoencoder.l1_sparsity(z).item()
    l0 = SparseAutoencoder.l0_sparsity(z).item()

    metrics[name] = {
        "mse": mse,
        "l1_sparsity": l1,
        "l0_sparsity": l0
    }

    print(f"MSE: {mse:.6f}")
    print(f"L1 sparsity: {l1:.6f}")
    print(f"L0 sparsity: {l0:.6f}")
    print("-" * 40)

print("\nМетрики рассчитаны")


## 7_Извлечение латентов для каждого датасета

In [ ]:
print("Извлечение латентных представлений SAE...")

latents = {}

for name in DATASETS:
    x = activations[name].detach().clone().float()
    _, z = models[name](x)

    latents[name] = z

    print(f"{name}: latents shape = {z.shape}")

print("\nЛатентные представления готовы")


## 8_Базовая статистика латентов

In [ ]:
print("Статистика латентных признаков...\n")

for name in DATASETS:
    z = latents[name]

    print(f"Dataset: {name}")
    print("Min:", z.min().item())
    print("Max:", z.max().item())
    print("Mean:", z.mean().item())
    print("Non-zero ratio:", (z > 0).float().mean().item())
    print("-" * 40)


## 9_Самые активные латенты

In [ ]:
print("Поиск наиболее активных SAE-фич...\n")

top_features = {}

for name in DATASETS:
    z = latents[name]

    mean_activation = z.mean(dim=0)
    topk = torch.topk(mean_activation, k=20)

    top_features[name] = topk.indices

    print(f"{name}: топ-20 наиболее активных фич:")
    print(topk.indices.tolist())
    print("-" * 40)


## 10_Наименее активные признаки

In [ ]:
print("Поиск наименее активных SAE-фич...\n")

rare_features = {}

for name in DATASETS:
    z = latents[name]

    mean_activation = z.mean(dim=0)
    bottomk = torch.topk(mean_activation, k=20, largest=False)

    rare_features[name] = bottomk.indices

    print(f"{name}: топ-20 наименее активных фич:")
    print(bottomk.indices.tolist())
    print("-" * 40)


## 11_Дополнитеьлные функции

In [ ]:
# Примеры для выбранной фичи
def show_top_examples(name, feature_index, k=10):
    print(f"\nТоп-{k} примеров для фичи {feature_index} в датасете {name}")

    z = latents[name]
    values = z[:, feature_index]

    topk = torch.topk(values, k=k)

    print("Значения активации:", topk.values.tolist())
    print("Индексы примеров:", topk.indices.tolist())


## 12_Демонстрация

In [ ]:
# Демонстрация
for name in DATASETS:
    print("\n==============================")
    print("Dataset:", name)

    f = top_features[name][0].item()
    show_top_examples(name, f)

## 13_Сохранение анализа

In [ ]:
import json

analysis = {
    name: {
        "top_features": top_features[name].tolist(),
        "rare_features": rare_features[name].tolist()
    }
    for name in DATASETS
}

out = GOOGLE_ROOT / "results" / "sae_feature_analysis.json"

with open(out, "w") as f:
    json.dump(analysis, f, indent=4)

print("Анализ SAE-фич сохранён в:", out)


## 14_Подготовка CLIP-обёртки

In [ ]:
from sae_clip.models.clip_wrapper import CLIPWrapper

print("Загрузка CLIP-модели...")

clip = CLIPWrapper()

print("CLIP готов")


# ГЛАВА 5 – БАЗОВАЯ ОЦЕНКА

## 1_Получение текстовых эмбеддингов классов

In [ ]:
print("Генерация текстовых эмбеддингов...")

text_embeddings = {}

for name in DATASETS:
    dataset = datasets[name]

    class_names = list(dataset.classes)

    print(f"\nДатасет: {name}")
    print(f"Количество классов: {len(class_names)}")

    emb = clip.encode_text(class_names)
    emb = emb.detach().clone().float()

    text_embeddings[name] = emb

    print(f"{name}: получено {emb.shape[0]} текстовых эмбеддингов, размерность {emb.shape[1]}")


## 2_Zero-shot baseline CLIP

In [ ]:
def get_labels(dataset):
    # для CIFAR10ZeroShotDataset
    if hasattr(dataset, "labels"):
        return dataset.labels

    # для Food101ZeroShotDataset (обёртка над torchvision)
    if hasattr(dataset, "ds"):
        if hasattr(dataset.ds, "targets"):
            return dataset.ds.targets
        if hasattr(dataset.ds, "_labels"):
            return dataset.ds._labels

    raise ValueError("Не удалось определить метки датасета")


## 3_Zero-shot оценка на исходных CLIP-активациях

In [ ]:
import torch

print("Zero-shot оценка на исходных CLIP-активациях...\n")

baseline_results = {}

device = torch.device("cpu")   # работаем в одном устройстве

for name in DATASETS:
    print(f"Оценка для датасета: {name}")

    labels = torch.tensor(get_labels(datasets[name]), dtype=torch.long, device=device)

    x = activations[name][:len(labels)].detach().clone().float().to(device)
    t = text_embeddings[name].detach().clone().float().to(device)

    logits = x @ t.T
    preds = logits.argmax(dim=1)

    acc = (preds == labels).float().mean().item()

    baseline_results[name] = acc

    print(f"{name}: baseline accuracy = {acc:.4f}")
    print("-" * 40)


## 4_Zero-shot оценка с использованием SAE

In [ ]:
print("Zero-shot оценка с использованием SAE...\n")

sae_results = {}

device = torch.device("cpu")

for name in DATASETS:
    print(f"Оценка для датасета: {name}")

    labels = torch.tensor(get_labels(datasets[name]), dtype=torch.long, device=device)

    x = activations[name][:len(labels)].detach().clone().float().to(device)

    model = models[name].to(device)
    x_hat, _ = model(x)

    t = text_embeddings[name].detach().clone().float().to(device)

    logits = x_hat @ t.T
    preds = logits.argmax(dim=1)

    acc = (preds == labels).float().mean().item()

    sae_results[name] = acc

    print(f"{name}: SAE accuracy = {acc:.4f}")
    print("-" * 40)


## 5_Сравнение результатов

In [ ]:
print("\nСравнение результатов:\n")

for name in DATASETS:
    print(name)
    print(f"Baseline CLIP: {baseline_results[name]:.4f}")
    print(f"SAE-based:     {sae_results[name]:.4f}")
    print("=" * 40)

## 6_Запуск официальной оценки

In [ ]:
!python scripts/evaluate_sae_full.py \
  --data_root /content/yadisk/SAE_PROJECT/datasets \
  --food_samples 10000 \
  --batch_size 128 \
  --results_dir /content/drive/MyDrive/SAE_PROJECT/results_full

Я делелал на kaggle отчет ниже.

In [ ]:
import pandas as pd
from pathlib import Path

print("\n=== ЗАГРУЗКА ИТОГОВЫХ РЕЗУЛЬТАТОВ ===")

BASE = Path("/content/drive/MyDrive/SAE_PROJECT/results")

csv_path = BASE / "results.csv"
json_path = BASE / "results.json"

# Чтение данных
if csv_path.exists():
    df = pd.read_csv(csv_path)
    print("Результаты загружены из CSV:", csv_path)
elif json_path.exists():
    df = pd.read_json(json_path)
    print("Результаты загружены из JSON:", json_path)
else:
    raise FileNotFoundError("Не найден results.csv или results.json")

print("\nИсходные данные:")
display(df)

# Приведение к формату таблицы SUMMARY
print("\n" + "="*60)
print("SUMMARY:")
print("="*60)

# Печать в текстовом формате
for _, row in df.iterrows():
    dataset = row.get("dataset", row.get("Dataset", "unknown"))
    method  = row.get("method", row.get("Method", "unknown"))
    acc     = row.get("accuracy", row.get("Accuracy", 0))
    note    = row.get("note", row.get("Note", ""))

    print(f"{dataset:<20} | {method:<12} | {acc:.4f} | {note}")

print("\n")

# Дополнительно: формируем таблицу для отображения
pretty = df.copy()

# Переименование колонок
rename_map = {}
for c in pretty.columns:
    rename_map[c] = c.capitalize()

pretty = pretty.rename(columns=rename_map)

print("Таблица результатов:")
display(pretty)


# ГЛАВА 6 – АНАЛИЗ SAE-ФИЧ

## 1_Извлечение топ-активаций

In [ ]:
print("\n=== Анализ SAE-фич для CIFAR-10 ===")

!python scripts/analyze_sae_features.py \
  --activations_path /content/drive/MyDrive/SAE_PROJECT/activations/activations_cifar10_vit_b32.pt \
  --sae_path /content/drive/MyDrive/SAE_PROJECT/models/sae_cifar_512d/sae_epoch_50.pt \
  --top_k 20 \
  --save_dir /content/drive/MyDrive/SAE_PROJECT/results/cifar10/sae_features


In [ ]:
print("\n=== Анализ SAE-фич для FOOD-101 ===")

!python scripts/analyze_sae_features.py \
  --activations_path /content/drive/MyDrive/SAE_PROJECT/activations/activations_food101_vit_b32.pt \
  --sae_path /content/drive/MyDrive/SAE_PROJECT/models/sae_food101_512d/sae_epoch_50.pt \
  --top_k 20 \
  --save_dir /content/drive/MyDrive/SAE_PROJECT/results/food101/sae_features



In [ ]:
# Интрепритация
INTERPRET_DATASETS = ["cifar10", "food101"]

## 2_Генерация коллажей (по каждому датасету)

In [ ]:
from pathlib import Path

print("\n=== ПРОВЕРКА НАЛИЧИЯ КОЛЛАЖЕЙ ДЛЯ CIFAR И FOOD ===")

BASE_ROOT = "/content/drive/MyDrive/SAE_PROJECT/results"
EXPECTED = 300


def check_dataset(ds):
    viz_dir = Path(f"{BASE_ROOT}/{ds}/feature_viz")
    print(f"\nПроверка датасета: {ds}")
    print("Папка:", viz_dir)

    if not viz_dir.exists():
        print("✘ Папка отсутствует")
        return False

    pngs = list(viz_dir.glob("feature_*.png"))
    print(f"Найдено коллажей: {len(pngs)} из ожидаемых {EXPECTED}")

    if len(pngs) >= EXPECTED:
        print("✔ Коллажи готовы")
        return True
    else:
        print("✘ Недостаточно коллажей")
        return False


ready_cifar = check_dataset("cifar10")
ready_food  = check_dataset("food101")

print("\nИТОГ ПРОВЕРКИ:")
print("CIFAR10 готов:", ready_cifar)
print("FOOD101 готов:", ready_food)


def generate_for(ds):
    print(f"\n=== ГЕНЕРАЦИЯ КОЛЛАЖЕЙ ДЛЯ {ds.upper()} ===")

    BASE = f"{BASE_ROOT}/{ds}"
    viz_dir = Path(f"{BASE}/feature_viz")

    if ds == "food101":
        script = "visualize_sae_features_food101.py"
        dataset_root = "/content/yadisk/SAE_PROJECT/datasets/food-101/images"
    else:
        script = "visualize_sae_features.py"
        dataset_root = "/content/yadisk/SAE_PROJECT/datasets"

    csv_path = f"{BASE}/sae_features/sae_topk_activations.csv"

    print("Скрипт:", script)
    print("CSV:", csv_path)
    print("Dataset root:", dataset_root)

    !python scripts/{script} \
      --csv_path {csv_path} \
      --dataset_root {dataset_root} \
      --features 300 \
      --grid 3 \
      --save_dir {viz_dir}

    print(f"\nГенерация завершена для: {ds}")


# --- ГЛАВНАЯ ЛОГИКА ---

if ready_cifar and ready_food:
    print("\n Коллажи готовы и для CIFAR, и для FOOD.")
    print("Генерация полностью пропускается.")

elif not ready_cifar and not ready_food:
    print("\n Коллажи отсутствуют для обоих датасетов.")
    print("Будет выполнена генерация для CIFAR и FOOD.")
    generate_for("cifar10")
    generate_for("food101")

elif not ready_cifar:
    print("\n Коллажи отсутствуют только для CIFAR10.")
    print("Будет выполнена генерация только для CIFAR10.")
    generate_for("cifar10")

elif not ready_food:
    print("\n Коллажи отсутствуют только для FOOD101.")
    print("Будет выполнена генерация только для FOOD101.")
    generate_for("food101")


# ГЛАВА 7 – АВТОИНТЕРПРЕТАЦИЯ

In [ ]:
# запускаем автоматическую интерпретацию фич SAE с помощью визуальных промптов и
# VLM (OpenRouter / BLIP‑2) по коллажам картинок для каждой фичи
# на этапе 1 интерпретация фич SAE идёт через внешнюю сильную VLM (через OpenRouter, хорошо понимающую русский/английский и сложные инструкции);
# когда лимит/токены кончились, скрипт автоматически переключается в режим локальной BLIP‑2 (модель для image→text, без интернета), но ей лучше давать промпты на английском;
# итог — для каждой фичи SAE получается автоматически сгенерированное человеческое текстовое описание визуального паттерна, который эта фича ловит
# sae_auto_interpretations.csv - дописываемая!!!!
# т.к. я потратил уже весь лимит на экспериментах, за денги решил через OpenRouter
# (model openai/gpt-4o-mini) выполнить автоматическую интерпретаци только 200 вместо 300 (50 русских + 50 ангискийх )

#     рекомендованный промт
#   промт для CIFAR10
#"Опишите в одном кратком предложении на русском языке, какая визуальная концепция является общей для всех изображений в этом коллаже. Избегайте слов "коллаж", "изображения", "разнообразие", "разнообразный"."
#"Describe in one concise English sentence what visual concept is common across all images in this collage. Avoid the words collage, images, variety, various."
#   промт для FOOD-101
# "Identify ONE shared food-related concept. Answer in a short English phrase (4–8 words). Avoid the words collage, images, variety, various. Focus on a concrete dish, ingredient, cooking method, or texture."
# "Определите ОДНО общее понятие, связанное с едой. Ответьте короткой фразой на русском языке (4-8 слов). Избегайте слов "коллаж", "изображения", "разнообразие", "разнообразный". Сосредоточьтесь на конкретном блюде, ингредиенте, способе приготовления или текстуре.

## 1_Выбор датасета и языка анализа

In [ ]:
# Выбор датасета и языка анализа
def choose_option(title, options):
    print(title)
    for k, v in options.items():
        print(f"{k} - {v}")
    choice = input("Введите номер: ").strip()
    if choice not in options:
        raise ValueError("Неверный выбор")
    return options[choice]

DATASET = choose_option(
    "Выберите датасет:",
    {"1": "cifar10", "2": "food101"}
)

LANG = choose_option(
    "Выберите язык анализа:",
    {"1": "en", "2": "ru"}
)

print("\nИтог:")
print("DATASET =", DATASET)
print("LANG =", LANG)


## 2_Ввод промпта

In [ ]:
print("\n=== ВВОД ПРОМПТОВ ДЛЯ ДВУЯЗЫЧНОЙ ИНТЕРПРЕТАЦИИ ===")

print("\nТекущий датасет:", DATASET)

# ---- Промпт по умолчанию (EN) ----
default_en = (
"Describe in one concise English sentence what visual concept is common "
"across all images in this collage."
)

print("\nПромпт по умолчанию (EN):")
print(default_en)
print("\nВведите свой английский промпт или нажмите Enter для использования стандартного.")

user_en = input("PROMPT EN: ").strip()

if user_en == "":
    PROMPT_EN = default_en
else:
    PROMPT_EN = user_en


# ---- Промпт по умолчанию (RU) ----
default_ru = (
"Опишите в одном кратком предложении на русском языке, какая визуальная "
"концепция является общей для всех изображений в этом коллаже."
)

print("\nПромпт по умолчанию (RU):")
print(default_ru)
print("\nВведите свой русский промпт или нажмите Enter для использования стандартного.")

user_ru = input("PROMPT RU: ").strip()

if user_ru == "":
    PROMPT_RU = default_ru
else:
    PROMPT_RU = user_ru


print("\n=== ИСПОЛЬЗУЕМЫЕ ПРОМПТЫ ===")
print("\n[EN]:", PROMPT_EN)
print("\n[RU]:", PROMPT_RU)


## 3_Генерация английских описаний

In [ ]:
# Генерация английских описаний (Автоинтерпретация на английском языке)
import os
from getpass import getpass

print("\n=== АВТОИНТЕРПРЕТАЦИЯ (ENGLISH) ===")

key = getpass("Введите OPENROUTER_API_KEY (EN): ")
os.environ["OPENROUTER_API_KEY"] = key

try:
    !python scripts/auto_interpret_sae_universal.py \
      --images_dir /content/drive/MyDrive/SAE_PROJECT/results/{DATASET}/feature_viz \
      --num_features 50 \
      --save_csv /content/drive/MyDrive/SAE_PROJECT/results/{DATASET}/interpretations_en.csv \
      --model openai/gpt-4o-mini \
      --prompt "{PROMPT_EN}"
finally:
    del os.environ["OPENROUTER_API_KEY"]
    del key


## 4_Генерация русских описаний

In [ ]:
# Генерация русских описаний (Автоинтерпретация на русском языке)
import os
from getpass import getpass

print("\n=== АВТОИНТЕРПРЕТАЦИЯ (RUSSIAN) ===")

key = getpass("Введите OPENROUTER_API_KEY (RU): ")
os.environ["OPENROUTER_API_KEY"] = key

try:
    !python scripts/auto_interpret_sae_universal.py \
      --images_dir /content/drive/MyDrive/SAE_PROJECT/results/{DATASET}/feature_viz \
      --num_features 50 \
      --save_csv /content/drive/MyDrive/SAE_PROJECT/results/{DATASET}/interpretations_ru.csv \
      --model openai/gpt-4o-mini \
      --prompt "{PROMPT_RU}"
finally:
    del os.environ["OPENROUTER_API_KEY"]
    del key


## 5_Объединение RU и EN для сравнения

In [ ]:
# Объединение RU и EN для сравнения
import pandas as pd
from pathlib import Path

print("\n=== СОЗДАНИЕ ОБЪЕДИНЁННОГО ФАЙЛА ДЛЯ СРАВНЕНИЯ RU ↔ EN ===")

BASE = f"/content/drive/MyDrive/SAE_PROJECT/results/{DATASET}"

en_path = f"{BASE}/interpretations_en.csv"
ru_path = f"{BASE}/interpretations_ru.csv"

print("Файл EN:", en_path)
print("Файл RU:", ru_path)

en = pd.read_csv(en_path)
ru = pd.read_csv(ru_path)

# Объединяем по feature_id
bilingual = en.merge(ru, on="feature_id", suffixes=("_en", "_ru"))

bilingual_path = f"{BASE}/interpretations_bilingual.csv"
bilingual.to_csv(bilingual_path, index=False)

print("\nОбъединённый файл сохранён:")
print(bilingual_path)
print("Размер таблицы:", bilingual.shape)

print("\nПример строк:")
display(bilingual.head(10))


# ГЛАВА 8 – ОЧИСТКА И АНАЛИЗ

## 1_Выбор языка для анализа

In [ ]:
# Выбор языка для анализа
print("\n=== НАСТРОЙКА АНАЛИЗА ===")
print("Текущий датасет:", DATASET)
print("Текущий язык анализа:", LANG)

BASE = f"/content/drive/MyDrive/SAE_PROJECT/results/{DATASET}"

INPUT = f"{BASE}/interpretations_{LANG}.csv"
CLEAN = f"{BASE}/interpretations_clean_{LANG}.csv"
LOG_CLEAN = f"{BASE}/cleaning_{LANG}.log"
LOG_ANALYZE = f"{BASE}/analysis_{LANG}.log"
REPORT = f"{BASE}/final_report_{LANG}.txt"

print("\nФайлы для анализа:")
print("Входной файл:", INPUT)
print("Очищенный файл:", CLEAN)
print("Отчёт:", REPORT)


## 2_Очистка интерпретаций

In [ ]:
# Очистка интерпретаций
print("\n=== ОЧИСТКА ИНТЕРПРЕТАЦИЙ ===")

!python scripts/clean_interpretations.py \
  --input_csv "{INPUT}" \
  --output_csv "{CLEAN}" \
  --log_file "{LOG_CLEAN}"


## 3_Анализ выбранного языка

In [ ]:
# Анализ выбранного языка
print("\n=== АНАЛИЗ ИНТЕРПРЕТАЦИЙ ===")

!python scripts/analyze_clean_interpretations.py \
  --input_csv "{CLEAN}" \
  --log_file "{LOG_ANALYZE}"


## 4_Формирование финального отчёта

In [ ]:
# Формирование финального отчёта
print("\n=== ФОРМИРОВАНИЕ ОТЧЁТА ===")

!python scripts/report_clean_interpretations.py \
  --input_csv "{CLEAN}" \
  --output_report "{REPORT}"

print("\nОтчёт сохранён:", REPORT)


# ГЛАВА 9 – ФИНАЛЬНАЯ ВИЗУАЛИЗАЦИЯ

## 1_Сравнение RU vs EN с коллажами

In [ ]:
# Просмотр результатов анализа
import pandas as pd

print("\n=== ИТОГОВАЯ ТАБЛИЦА (язык анализа:", LANG, ") ===")

df = pd.read_csv(CLEAN)
display(df.head(20))


## 2_Сравнение RU vs EN с коллажами с визуализацией

In [ ]:
compare = bilingual[["feature_id", "interpretation_en", "interpretation_ru"]]


In [ ]:
import pandas as pd
from IPython.display import Image, display, HTML
from pathlib import Path

print("\n=== ВИЗУАЛЬНАЯ ТАБЛИЦА СРАВНЕНИЯ RU ↔ EN С КОЛЛАЖАМИ ===")

BASE = f"/content/drive/MyDrive/SAE_PROJECT/results/{DATASET}"

bilingual = pd.read_csv(f"{BASE}/interpretations_bilingual.csv")

viz_dir = Path(f"{BASE}/feature_viz")

def find_image(feature_id):
    # Формат ваших файлов: feature_0000.png
    fname = f"feature_{int(feature_id):04d}.png"
    path = viz_dir / fname

    if path.exists():
        return path
    return None


def show_feature(row):
    fid = row["feature_id"]
    en = row["interpretation_en"]
    ru = row["interpretation_ru"]

    html = f"""
    <h3>Feature {fid}</h3>
    <b>EN:</b> {en}<br>
    <b>RU:</b> {ru}<br><br>
    """

    display(HTML(html))

    img = find_image(fid)

    if img:
        display(Image(filename=str(img), width=300))
    else:
        print(f"Коллаж не найден: feature_{int(fid):04d}.png")


N = 20

for _, row in bilingual.head(N).iterrows():
    show_feature(row)


## 3_Генерация HTML-ОТЧЁТА

In [ ]:
import pandas as pd
from pathlib import Path

print("\n=== ГЕНЕРАЦИЯ ПОЛНОГО HTML-ОТЧЁТА ===")

BASE = f"/content/drive/MyDrive/SAE_PROJECT/results/{DATASET}"

bilingual_path = f"{BASE}/interpretations_bilingual.csv"
viz_dir = Path(f"{BASE}/feature_viz")

df = pd.read_csv(bilingual_path)

def image_tag(feature_id):
    fname = f"feature_{int(feature_id):04d}.png"
    path = viz_dir / fname

    if path.exists():
        return f'<img src="feature_viz/{fname}" width="280">'
    else:
        return "<i>image not found</i>"


html_parts = []

# ---- Заголовок отчёта ----
html_parts.append(f"""
<html>
<head>
<meta charset="utf-8">
<title>SAE Interpretation Report - {DATASET}</title>
<style>
body {{ font-family: Arial; margin: 30px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ccc; padding: 10px; vertical-align: top; }}
th {{ background-color: #f0f0f0; }}
img {{ border: 1px solid #aaa; }}
h1, h2 {{ color: #333; }}
.prompt {{ background: #f9f9f9; padding: 10px; border: 1px dashed #aaa; }}
</style>
</head>
<body>
""")

html_parts.append(f"<h1>SAE Feature Interpretations Report</h1>")
html_parts.append(f"<h2>Dataset: {DATASET}</h2>")

# ---- Блок с промптами ----
html_parts.append("<h2>Used Prompts</h2>")

html_parts.append("<h3>English Prompt</h3>")
html_parts.append(f"<div class='prompt'>{PROMPT_EN}</div>")

html_parts.append("<h3>Russian Prompt</h3>")
html_parts.append(f"<div class='prompt'>{PROMPT_RU}</div>")

html_parts.append("<hr>")

# ---- Таблица с результатами ----
html_parts.append("<h2>Feature Interpretations</h2>")

html_parts.append("""
<table>
<tr>
<th>Feature ID</th>
<th>Collage</th>
<th>English Interpretation</th>
<th>Russian Interpretation</th>
</tr>
""")

for _, row in df.iterrows():
    fid = row["feature_id"]
    en = row["interpretation_en"]
    ru = row["interpretation_ru"]

    img_html = image_tag(fid)

    html_parts.append(f"""
    <tr>
        <td><b>{fid}</b></td>
        <td>{img_html}</td>
        <td>{en}</td>
        <td>{ru}</td>
    </tr>
    """)

html_parts.append("</table>")
html_parts.append("</body></html>")

report_html = "\n".join(html_parts)

report_path = f"{BASE}/SAE_interpretation_report_{DATASET}.html"

with open(report_path, "w", encoding="utf-8") as f:
    f.write(report_html)

print("\nОтчёт успешно создан:")
print(report_path)
